In [14]:
#Libraries
import numpy as np
import numpy.ma as ma
from scipy import ndimage
from skimage import exposure, transform, io
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import tifffile as tif
import PIL
import napari
from skimage.registration import phase_cross_correlation
from skimage.transform import AffineTransform, warp
from scipy.ndimage import fourier_shift,convolve
from PIL import Image
#
import skimage
import imageio

In [15]:
# Import and inspect the image data
# Start coding here
filepath ='IPQDA_23_ASS_B_data/W47-SGFP2-mScarlet-I-01-1_2channels.tif'
image = tif.imread(filepath)

# Start coding here
donor_channel = image[:, 0]
acceptor_channel = image[:, 1]
# End coding here

#Calclate and plot the z-projections (use mean) of both channels
# Start coding here
z_project_donor = np.mean(image[:, 0], axis=0)
z_project_acceptor = np.mean(image[:, 1], axis=0)
# End coding here

# Plot z-projections
#plt.imshow(z_project_donor)
#plt.show()

#plt.imshow(z_project_acceptor)
#plt.show()
# Plot the z-projections
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(z_project_donor, cmap='gray')
plt.title('Z-Projection (Donor Channel)')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(z_project_acceptor, cmap='gray')
plt.title('Z-Projection (Acceptor Channel)')
plt.axis('off')

plt.tight_layout()


In [16]:
mean_bg=float(250.49)
std_bg=float(129.47)
threshold = mean_bg - std_bg
background_mask = z_project_donor < threshold

z_project_donor[background_mask] = 0
z_project_acceptor[background_mask]=0

In [17]:
mean_donor=[]
mean_acceptor=[]
stddev_donor=[]
stddev_acceptor=[]
#z_project_donor = np.mean(image[:, 0], axis=0)


for t in range(len(donor_channel)):
    mean_d = np.mean(donor_channel[t])
    stddev_d = np.std(donor_channel[t])
    mean_a = np.mean(acceptor_channel[t])
    stddev_a = np.std(acceptor_channel[t])
    mean_donor.append(mean_d)
    mean_acceptor.append(mean_a)
    stddev_donor.append(stddev_d)
    stddev_acceptor.append(stddev_a)


In [18]:
#Plotting
plt.plot(mean_donor,label='donor')
plt.plot(mean_acceptor,label='acceptor')
# Set the labels
plt.xlabel('time')
plt.ylabel('intensity')
# Set the legend
plt.legend()
#plt.show()#####·········································
plt.savefig('IPQDA_B_mean_time.png')
plt.close()

plt.plot(stddev_donor,label='donor')
plt.plot(stddev_acceptor,label='acceptor')
plt.xlabel('time')
plt.ylabel('intensity')

# Set the legend
plt.legend()

In [10]:
#Image processing steps
#Initializing parameters
# Define a smoothing kernel
ks = (3,3) # kernel size should be a tuple
kernel = np.ones(ks) / np.prod(ks) # create box filter kernel

# Initialize output array
corrected_frames = np.zeros_like(image,dtype=np.float64)

#Preparing the image data
# Convert pixel type to float
image = image.astype(np.float64)

# Channels extraction
donor_channel = image[:,0,:,:]
acceptor_channel = image[:,1,:,:]
#Now they begin - image processing
#image[time,donor/acceptor, X, Y]


In [11]:
for t in range(image.shape[0]):

    #######################################
    ##Background substraction
    #Start coding here
    #z_project_donor[background_mask] = 0
    #projections [X,Y]
    #xqxq=donor_channel[t, :, :] # shape [X,Y]
    #background_mask [X, Y]
    donor_channel[t, :, :][background_mask] = 0
    acceptor_channel[t, :, :][background_mask] = 0

    #######################################
    ##Image registration
    #Calculate the shift between the two images
    shift, error, diffphase = phase_cross_correlation(donor_channel[t, :, :],acceptor_channel[t, :, :])

    #Create an affine transform object with the shift
    tform = AffineTransform(translation=(shift[0],-shift[1]))

    #Apply the transformation to the image
    corrected_image = warp(acceptor_channel[t, :, :], tform.inverse)

    # Replace the original channel with the corrected channel
    acceptor_channel[t, :, :] = corrected_image

    #######################################
    ##Image processing steps to reduce noise
    ##Smoothing (hint: use the created kernel)
    donor_channel[t, :, :] = convolve(donor_channel[t, :, :], kernel)
    acceptor_channel[t, :, :] = convolve(acceptor_channel[t, :, :], kernel)

    #Thresholding so that pixels with low signal are assigned with np.nan.
    #Use the background mean and standard deviation intensity in this step to form a threshold value

    #Well done. Let's keep the corrected frames
    corrected_frames[t, 1, :, :] = acceptor_channel[t, :, :]
    corrected_frames[t, 0, :, :] = donor_channel[t, :, :]
    print(f'{t} of {image.shape[0]}')

    # End coding here

0 of 70
1 of 70
2 of 70
3 of 70
4 of 70
5 of 70
6 of 70
7 of 70
8 of 70
9 of 70
10 of 70
11 of 70
12 of 70
13 of 70
14 of 70
15 of 70
16 of 70
17 of 70
18 of 70
19 of 70
20 of 70
21 of 70
22 of 70
23 of 70
24 of 70
25 of 70
26 of 70
27 of 70
28 of 70
29 of 70
30 of 70
31 of 70
32 of 70
33 of 70
34 of 70
35 of 70
36 of 70
37 of 70
38 of 70
39 of 70
40 of 70
41 of 70
42 of 70
43 of 70
44 of 70
45 of 70
46 of 70
47 of 70
48 of 70
49 of 70
50 of 70
51 of 70
52 of 70
53 of 70
54 of 70
55 of 70
56 of 70
57 of 70
58 of 70
59 of 70
60 of 70
61 of 70
62 of 70
63 of 70
64 of 70
65 of 70
66 of 70
67 of 70
68 of 70
69 of 70


In [12]:
#Plotting
plt.imshow(corrected_frames[12, 0,:,:])

In [ ]:
# Calculate FRET ratios. R = Acceptor/Donor
# Start coding here
print(corrected_frames.shape)
ratio = corrected_frames[:,1,:,:]/corrected_frames[:,0,:,:]
#print(ratio)
"""
ratio=[]
for t in range(image.shape[0]):
    r=corrected_frames[t, 1, :, :] / corrected_frames[t, 0, :, :]
    #plt.imshow(r)
    #plt.show()
    ratio.append(r)
ratio = np.stack(ratio)
# End coding here
"""
"""
frames = []
for i in range(ratio.shape[0]):
    # Create a PIL Image object from the current frame
    frame = Image.fromarray(ratio[i])
    frames.append(frame)

frames[0].save("ratio.gif", format="GIF", append_images=frames[1:], save_all=True, duration=200, loop=0)
"""
#Plot a layer
plt.imshow(ratio[12,:,:])
#plt.show() #####·········································
# Export the FRET ratios images as an animated gif
# Start coding here
imageio.mimsave('fret_ratios.gif', ratio, format='GIF',duration=0.2)